In [1]:
import numpy as np
from matplotlib import pyplot as plt

from keras.layers import Dense, Input
from keras.models import Model

import omnifold as of
import os
import tensorflow as tf
import pandas as pd

import energyflow as ef
import energyflow.archs

/Volumes/WorkDrive/STAR-Workspace/D0Analysis/Analysis/AuAu/CombinedAnalysisSuite/MultiFold/multifold/lib/python3.11/site-packages/energyflow/archs/__init__.py:30: UserWarning: could not import some architectures - cannot import name '__version__' from 'tensorflow.keras' (/Volumes/WorkDrive/STAR-Workspace/D0Analysis/Analysis/AuAu/CombinedAnalysisSuite/MultiFold/multifold/lib/python3.11/site-packages/keras/api/_v2/keras/__init__.py)
  warnings.warn('could not import some architectures - ' + str(e))


In [4]:
N = 10**5

In [5]:
#Synthetic
X1 = np.random.normal(0.2,0.8,N)  # Generator-level synthetic sample
Y1 = np.array([(x + np.random.normal(0, 0.5)) for x in X1])  # Detector smearing for synthetic sample

X2 = 1 - np.random.power(5, N)
Y2 = np.array([(x + np.random.normal(0, 0.9)) for x in X2])  # Detector smearing for synthetic sample

X3 = np.random.power(8, N)
Y3 = np.array([(x + np.random.normal(0, 0.3)) for x in X2])  # Detector smearing for synthetic sample

X = [X1, X2, X3]
Y = [Y1, Y2, Y3]

In [6]:
#Data
A1 = np.random.normal(0,1,N)  # Generator-level synthetic sample
B1 = np.array([(x + np.random.normal(0, 0.5)) for x in X1])  # Detector smearing for synthetic sample

A2 = 1 - np.random.power(5.5, N)
B2 = np.array([(x + np.random.normal(0, 0.9)) for x in X2])  # Detector smearing for synthetic sample

A3 = np.random.power(7.5, N)
B3 = np.array([(x + np.random.normal(0, 0.3)) for x in X2])  # Detector smearing for synthetic sample

A = np.array([A1, A2, A3])
B = np.array([B1, B2, B3])

In [7]:
obs_multifold = ['P1', 'P2', 'P3']
obs = {}
[obs.setdefault(x, {}).update({'func': lambda dset: np.asarray(dset)}) for x in obs_multifold]
obs.items()

dict_items([('P1', {'func': <function <listcomp>.<lambda> at 0x284edee80>}), ('P2', {'func': <function <listcomp>.<lambda> at 0x28549c400>}), ('P3', {'func': <function <listcomp>.<lambda> at 0x28549c220>})])

In [8]:
i = 0
for obkey, ob in obs.items():
    ob['genobs'], ob['simobs'], ob['truobs'], ob['dataobs'] = ob['func'](X[i]), ob['func'](Y[i]), ob['func'](A[i]), ob['func'](B[i])
    i+=1
    print('Done with', obkey)

Done with P1
Done with P2
Done with P3


In [9]:
# set up the array of data/simulation detector-level observables
X_det = np.asarray([np.concatenate(
    (obs[obkey]['dataobs'], obs[obkey]['simobs'])) for obkey in obs_multifold]).T
Y_det = pd.get_dummies(np.concatenate((np.ones(len(obs['P1']['dataobs'])),
                                                np.zeros(len(obs['P1']['simobs'])))), dtype=int).to_numpy()

In [10]:
# set up the array of generation particle-level observables
X_gen = np.asarray([np.concatenate(
    (obs[obkey]['genobs'], obs[obkey]['genobs'])) for obkey in obs_multifold]).T
Y_gen = pd.get_dummies(np.concatenate((np.ones(len(obs['P1']['genobs'])),
                                                np.zeros(len(obs['P1']['genobs'])))), dtype=int).to_numpy()

In [11]:
# standardize the inputs
X_det = (X_det - np.mean(X_det, axis=0))/np.std(X_det, axis=0)
X_gen = (X_gen - np.mean(X_gen, axis=0))/np.std(X_gen, axis=0)

In [12]:
# Specify the training parameters
# model parameters for the Step 1 network
model_layer_sizes = [100, 100]  # use this for the full network size
# model_layer_sizes = [100, 100, 100]  # use this for the full network size
det_args = {'input_dim': len(obs_multifold), 'dense_sizes': model_layer_sizes,
            'patience': 10, 'filepath': 'Step1_{}', 'save_weights_only': False,
            'modelcheck_opts': {'save_best_only': True, 'verbose': 1}}
# model parameters for the Step 2 network
mc_args = {'input_dim': len(obs_multifold), 'dense_sizes': model_layer_sizes,
           'patience': 10, 'filepath': 'Step2_{}', 'save_weights_only': False,
           'modelcheck_opts': {'save_best_only': True, 'verbose': 1}}
# general training parameters
fitargs = {'batch_size': 500, 'epochs': 2, 'verbose': 1}
# reweight the sim and data to have the same total weight to begin with
ndata, nsim = np.count_nonzero(Y_det[:,1]), np.count_nonzero(Y_det[:,0])
wdata = np.ones(ndata)
winit = ndata/nsim*np.ones(nsim)

In [13]:
# apply the OmniFold procedure to get weights for the generation
multifold_ws = of.omnifold(X_gen, Y_gen, X_det, Y_det, wdata, winit,
                                (ef.archs.DNN, det_args), (ef.archs.DNN, mc_args),
                                fitargs, val=0.2, it=itnum, trw_ind=-2, weights_filename='Test')

AttributeError: module 'energyflow.archs' has no attribute 'DNN'